# Valuing European-style Swaptions with QL Example

We value a European swaption using Black's model and try to replicate a QL example at http://khandrikacm.blogspot.com/2014/03/european-style-interest-rate-swaption.html

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from financepy.products.libor import *
from financepy.finutils import *
from financepy.market.curves import *
from financepy.models.FinModelBlack import FinModelBlack

###################################################################
# FINANCEPY BETA Version 0.180 - This build: 22 Sep 2020 at 19:38 #
#     This software is distributed FREE & WITHOUT ANY WARRANTY    #
# For info and disclaimer - https://github.com/domokane/FinancePy #
###################################################################



## Building a Libor Discount Curve

In [3]:
valuationDate = FinDate(28, 2, 2014)
settlementDate = FinDate(4, 3, 2014)

We have a vector of dates and zero rates

In [5]:
depoDCCType = FinDayCountTypes.THIRTY_E_360_ISDA
depos = []

depo = FinLiborDeposit(settlementDate, "1W", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "1M", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "3M", 0.0023, depoDCCType); depos.append(depo)
depo = FinLiborDeposit(settlementDate, "6M", 0.0023, depoDCCType); depos.append(depo)

# No convexity correction provided so I omit interest rate futures

swaps = []
swapType = FinLiborSwapTypes.PAYER
fixedDCCType = FinDayCountTypes.ACT_365F
fixedFreqType = FinFrequencyTypes.SEMI_ANNUAL

swap = FinLiborSwap(settlementDate, "3Y", swapType, 0.00790, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "4Y", swapType, 0.01200, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "5Y", swapType, 0.01570, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "6Y", swapType, 0.01865, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "7Y", swapType, 0.02160, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "8Y", swapType, 0.02350, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "9Y", swapType, 0.02540, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "10Y", swapType, 0.0273, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "15Y", swapType, 0.0297, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "20Y", swapType, 0.0316, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "25Y", swapType, 0.0335, fixedFreqType, fixedDCCType); swaps.append(swap)
swap = FinLiborSwap(settlementDate, "30Y", swapType, 0.0354, fixedFreqType, fixedDCCType); swaps.append(swap)

liborCurve = FinLiborCurve(settlementDate, depos, [], swaps)

In [7]:
exerciseDate = settlementDate.addTenor("5Y")
swapMaturityDate = exerciseDate.addTenor("5Y")

swapFixedCoupon = 0.040852
swapFixedFrequencyType = FinFrequencyTypes.SEMI_ANNUAL
swapFixedDayCountType = FinDayCountTypes.THIRTY_E_360_ISDA

swapFloatFrequencyType = FinFrequencyTypes.QUARTERLY
swapFloatDayCountType = FinDayCountTypes.ACT_360

swapNotional = ONE_MILLION
swapType = FinLiborSwapTypes.PAYER 
calendarType = FinCalendarTypes.TARGET
busDayAdjustType = FinBusDayAdjustTypes.MODIFIED_FOLLOWING
dateGenRuleType = FinDateGenRuleTypes.BACKWARD

In [8]:
swaption = FinLiborSwaption(settlementDate,
                            exerciseDate,
                            swapMaturityDate,
                            swapType,
                            swapFixedCoupon,
                            swapFixedFrequencyType,
                            swapFixedDayCountType, 
                            swapNotional, 
                            swapFloatFrequencyType,
                            swapFloatDayCountType,
                            calendarType, 
                            busDayAdjustType,
                            dateGenRuleType)

In [9]:
print(swaption)

OBJECT TYPE: FinLiborSwaption
SETTLEMENT DATE: TUE 04 MAR 2014
EXERCISE DATE: MON 04 MAR 2019
SWAP TYPE: FinLiborSwapTypes.PAYER
SWAP MATURITY DATE: MON 04 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FinFrequencyTypes.QUARTERLY
FLOAT DAY COUNT: FinDayCountTypes.ACT_360



In [10]:
model = FinModelBlack(0.1533)

In [11]:
swaption.value(settlementDate, liborCurve, model)

23165.010682062704

This is close to a value of 23162 using QL

## Internals

In [12]:
print(swaption)

OBJECT TYPE: FinLiborSwaption
SETTLEMENT DATE: TUE 04 MAR 2014
EXERCISE DATE: MON 04 MAR 2019
SWAP TYPE: FinLiborSwapTypes.PAYER
SWAP MATURITY DATE: MON 04 MAR 2024
SWAP NOTIONAL: 1000000
FIXED COUPON: 4.0852
FIXED FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
FLOAT FREQUENCY: FinFrequencyTypes.QUARTERLY
FLOAT DAY COUNT: FinDayCountTypes.ACT_360
PV01: 4.161685329450341
FWD SWAP RATE: 4.086140343654034
FWD DF TO EXPIRY: 0.9237933367937652


We can see that the forward swap rate almost equals the fixed coupon. The underlying swap is close to being ATM forward.

In [13]:
swaption.printSwapFixedLeg()

START DATE: MON 04 MAR 2019
MATURITY DATE: MON 04 MAR 2024
COUPON (%): 4.0852
FIXED LEG FREQUENCY: FinFrequencyTypes.SEMI_ANNUAL
FIXED LEG DAY COUNT: FinDayCountTypes.THIRTY_E_360_ISDA
VALUATION DATE TUE 04 MAR 2014
PAYMENT_DATE     YEAR_FRAC        FLOW         DF         DF*FLOW       CUM_PV
TUE 04 MAR 2014          -            -   1.00000000            -            -
WED 04 SEP 2019  0.5000000     20426.00   0.90872318     18561.58     18561.58
WED 04 MAR 2020  0.5000000     20426.00   0.89262979     18232.86     36794.44
FRI 04 SEP 2020  0.5000000     20426.00   0.87518398     17876.51     54670.94
THU 04 MAR 2021  0.5000000     20426.00   0.85693776     17503.81     72174.75
MON 06 SEP 2021  0.5055556     20652.96   0.84085474     17366.14     89540.89
FRI 04 MAR 2022  0.4944444     20199.04   0.82477877     16659.74    106200.63
MON 05 SEP 2022  0.5027778     20539.48   0.80762580     16588.21    122788.85
MON 06 MAR 2023  0.5027778     20539.48   0.79020647     16230.43    1390

In [14]:
swaption.printSwapFloatLeg()

START DATE: MON 04 MAR 2019
MATURITY DATE: MON 04 MAR 2024
SPREAD COUPON (%): 0.0
FLOAT LEG FREQUENCY: FinFrequencyTypes.QUARTERLY
FLOAT LEG DAY COUNT: FinDayCountTypes.ACT_360
VALUATION DATE TUE 04 MAR 2014
Floating Flows not calculated.


Copyright (c) 2020 Dominic O'Kane